<a href="https://colab.research.google.com/github/MeidanGR/GooglePlayStore_Preprocessing_Python/blob/main/GooglePlayStore_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Play Store Data Preprocessing**
This data preprocessing progress has been established for an academic project of a BI course. The data will be analyzed within the project.

Major thanks to **Gautham Prakash** for publishing & maintenance this dataset, which you can find at https://www.kaggle.com/gauthamp10/google-playstore-apps


# **Google auth & libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# **CSV LOAD**


In [11]:
#Loading csv
df = pd.read_csv('/content/drive/My Drive/Google-Playstore.csv')

print("Dataframe original shape is", np.shape(df), end='\n\n')
print(df.info())
df.sample(2)

Dataframe original shape is (603047, 23)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603047 entries, 0 to 603046
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   App Name           603046 non-null  object 
 1   App Id             603047 non-null  object 
 2   Category           603047 non-null  object 
 3   Rating             599287 non-null  float64
 4   Rating Count       599287 non-null  float64
 5   Installs           602875 non-null  object 
 6   Minimum Installs   602875 non-null  float64
 7   Maximum Installs   603047 non-null  int64  
 8   Free               603047 non-null  bool   
 9   Price              603047 non-null  float64
 10  Currency           602866 non-null  object 
 11  Size               603047 non-null  object 
 12  Minimum Android    602556 non-null  object 
 13  Developer Id       603045 non-null  object 
 14  Developer Website  423783 non-null  object 
 15  Developer

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
433649,Posture CHECKER plus,jp.co.rugle.postureplus,Health & Fitness,4.7,15.0,"1,000+",1000.0,1295,False,5.02,USD,5.9M,4.0.3 and up,MEDIC ENGINEERING,http://www.rugle.co.jp,support@rugle.co.jp,"Sep 9, 2013",15/01/2020 8:54,Everyone,http://www7b.biglobe.ne.jp/~mobileapps/files/p...,False,False,False
519734,Supercons Dark - The Superhero Icon Pack,com.supercons.dark,Personalization,4.3,66.0,"5,000+",5000.0,8583,False,0.99,USD,7.9M,4.0.3 and up,badpixel,NaN,rahullama34@gmail.com,"Feb 27, 2019",19/07/2019 21:59,Everyone,https://rahul3183.github.io,False,False,False


# **DATA FILTERING**
## Agenda: 
Data reduction down to ~100K rows, for a decent dataframe volume, shape, & a faster run in Google Data Studio.

## Guidelines:
- **Minimum Android** is the minimum android version which allowes operation for the app. The mose popular values were found: 4.1\4.0.3\6.0 and up & Varies with device.
- **Size** must exceed 1MB.
- **Currency** will be USD only.
- **Last updated** during 2020.


In [4]:
#Coloumn labels space fix
df.columns = [x.replace(" ", "_") for x in df.columns]

#Rows SELECT
df = df.query('(Minimum_Android == "4.1 and up" | Minimum_Android == "4.0.3 and up" | Minimum_Android == "Varies with device" | \
Minimum_Android == "6.0 and up") & not Size.str.endswith("k") & Currency == "USD" & Last_Updated.str.contains("2020")',engine='python')

#Coloumns FILTER
df = df.filter(items=['App_Name', 'Category', 'Rating', 'Rating_Count', 'Installs', 'Price', 'Size','Released','Last_Updated','Ad_Supported', 'In_App_Purchases'])

#Saving a copy of dataframe
dfcopy = df.copy(deep=True)

print("Dataframe reduced shape is", np.shape(df))
df.sample(10)

Dataframe reduced shape is (112092, 11)


,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
105549,Caramel Icon Pack,Personalization,4.2,17.0,100+,1.49,17M,"Apr 22, 2019",24/02/2020 5:23,False,False
58771,Asmaul husna,Books & Reference,5.0,328.0,"10,000+",0.00,12M,"Jul 29, 2019",24/05/2020 12:17,False,False
301368,Kiddos in Camp - Free Educational Games for Kids,Educational,4.4,91.0,"10,000+",0.00,34M,"Mar 6, 2020",15/06/2020 15:26,True,True
90280,Boom!,Tools,4.2,5.0,"1,000+",0.00,4.1M,"Jul 12, 2016",01/05/2020 7:15,True,True
374614,Napoleon Hill's Audio & E-Books,Books & Reference,4.6,150.0,"10,000+",0.00,40M,"Apr 12, 2019",10/01/2020 21:51,True,False
4248,1200 common english words with Hindi Meaning,Education,4.2,216.0,"50,000+",0.00,1.4M,"Oct 24, 2017",07/03/2020 0:34,True,False
13853,বিরিয়ানি রেসিপি~Biriyani Recipes,Food & Drink,0.0,0.0,50+,0.00,10.0M,"Oct 2, 2019",21/03/2020 0:36,True,False
130369,ConVos,Tools,NaN,NaN,5+,0.00,Varies with device,NaN,14/07/2020 19:10,False,False
568144,Ursinhos de Amor ❤️ Lindos Ursinhos Carinhosos,Entertainment,4.3,6.0,100+,0.00,9.2M,"Apr 4, 2019",23/05/2020 17:05,True,False
288124,Jamak Dan Qasar,Education,4.6,532.0,"100,000+",0.00,30M,"Mar 29, 2015",19/03/2020 17:35,True,False


# **CHECKPOINT DATA INFO**

In [ ]:
print("Dataframe NULL count:")
print(df.isnull().sum(axis = 0), end="\n\n")

df.info()

Dataframe NULL count:
App_Name               1
Category               0
Rating              1880
Rating_Count        1880
Installs               0
Price                  0
Size                   0
Released            6338
Last_Updated           0
Ad_Supported           0
In_App_Purchases       0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112092 entries, 20 to 603046
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112091 non-null  object 
 1   Category          112092 non-null  object 
 2   Rating            110212 non-null  float64
 3   Rating_Count      110212 non-null  float64
 4   Installs          112092 non-null  object 
 5   Price             112092 non-null  float64
 6   Size              112092 non-null  object 
 7   Released          105754 non-null  object 
 8   Last_Updated      112092 non-null  object 
 9   Ad_Supported      112092 non-null  bool   
 

In [ ]:
#Retrieve coloumn general info 
name = 'Category'
df[name].describe()

count        112092
unique           48
top       Education
freq          12333
Name: Category, dtype: object

# **DATA CLEAN & CORRECTION**
- A null value that was found in **App_Name** are not acceptable for furthor analysis.
- The null values in **Rating, Rating_Count** will be replaced with mean values.

In [5]:
#NULL Extraction
df = df.query('not App_Name.isnull()',engine='python')

# Function replaces np.nan values in all of the data frame with relevant statistics \ dal7collab
def rep_missings(data_frame):
    d = data_frame.copy(deep = True)
    for c in data_frame:
        if (data_frame[c].dtype == 'float64'):
            if data_frame[c].isnull().values.any() :
                m = data_frame[c].dropna().mean()
                d[c].fillna(m, inplace = True)
    return d;

# Activating the function
df = rep_missings(df)

In [7]:
#Coloumn types correction
df['App_Name'] = df['App_Name'].astype({'App_Name': 'string'})
df['Category'] = df['Category'].astype({'Category': 'string'})
df['Rating'] = df['Rating'].astype({'Rating': 'float32'})
df['Rating_Count'] = df['Rating_Count'].astype({'Rating_Count': 'int32'})
df['Installs'] = df['Installs'].astype({'Installs': 'string'})
df['Price'] = df['Price'].astype({'Price': 'float32'})
df['Size'] = df['Size'].astype({'Size': 'string'})
df['Released']= pd.to_datetime(df['Released']) 
df['Last_Updated']= pd.to_datetime(df['Last_Updated']) 

#'Size' coloumn excluding "Varies with device"
df['Size'] = df['Size'].replace(to_replace="Varies with device", 
                                   value = np.nan)


print("Dataframe final shape is", np.shape(df))
df.sample(10)

Dataframe final shape is (112091, 11)


,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
158829,Digital Stress Buddy,Medical,0.0,0,10+,0.0,5.8M,2020-05-06,2020-05-27 21:41:00,False,False
334414,Madrid Guía,Travel & Local,0.0,0,100+,0.0,4.2M,2018-12-10,2020-06-15 20:58:00,False,False
428573,Planfix Lite,Business,0.0,0,500+,0.0,5.8M,2020-06-20,2020-03-07 11:41:00,False,False
265373,예스24 eBook - YES24 eBook,Books & Reference,3.4,8763,"1,000,000+",0.0,3.7M,2015-05-31,2020-05-13 11:21:00,False,False
470017,Rope Master 3D,Puzzle,1.9,12,"5,000+",0.0,6.6M,2020-04-28,2020-05-18 16:45:00,True,False
524597,SZ Viewer A1: Diagnostics for Suzuki,Auto & Vehicles,4.8,1512,"50,000+",0.0,<NA>,2015-12-18,2020-07-05 04:38:00,False,False
317323,Learn French For Kids,Education,4.3,405,"50,000+",0.0,39M,2016-10-31,2020-04-27 16:57:00,True,False
575717,Viking Radio,Music & Audio,4.4,139,"10,000+",0.0,<NA>,NaT,2020-10-07 18:39:00,True,False
412924,Paithani Saree Style & Designs,Lifestyle,0.0,0,"1,000+",0.0,5.4M,2018-03-31,2020-06-24 10:42:00,True,False
322419,Libro de 2 Timoteo,Books & Reference,0.0,0,10+,0.0,4.0M,2020-03-23,2020-03-24 05:22:00,True,False


# **POSTPROCESSING DATA INFO**

In [8]:
#Genereal information after preprocessing
print(df.info(),end = '\n\n')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112091 entries, 20 to 603045
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   App_Name          112091 non-null  string        
 1   Category          112091 non-null  string        
 2   Rating            112091 non-null  float32       
 3   Rating_Count      112091 non-null  int32         
 4   Installs          112091 non-null  string        
 5   Price             112091 non-null  float32       
 6   Size              101269 non-null  string        
 7   Released          105753 non-null  datetime64[ns]
 8   Last_Updated      112091 non-null  datetime64[ns]
 9   Ad_Supported      112091 non-null  bool          
 10  In_App_Purchases  112091 non-null  bool          
dtypes: bool(2), datetime64[ns](2), float32(2), int32(1), string(4)
memory usage: 7.5 MB
None



# **EXPORT TO GOOGLE DRIVE**


In [9]:
#Exporting a csv into google drive
df.to_csv('/content/drive/My Drive/GooglePlay-Store_Optimized.csv')